In [2]:
%load_ext line_profiler

In [3]:
import pandas as pd

In [4]:
from tardis import run_tardis
sim = run_tardis('tardis_example.yml')

/home/jals/miniconda3/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


[tardis.plasma.standard_plasmas][INFO   ]  Reading Atomic Data from kurucz_cd23_chianti_H_He.h5 (standard_plasmas.py:74)
[tardis.io.atom_data.util][INFO   ]  Atom Data kurucz_cd23_chianti_H_He.h5 not found in local path. Exists in TARDIS Data repo /home/jals/Downloads/tardis-data/kurucz_cd23_chianti_H_He.h5 (util.py:29)
[py.warnings         ][WARNING]  /home/jals/miniconda3/envs/tardis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: PerformanceWarning: indexing past lexsort depth may impact performance.
  exec(code_obj, self.user_global_ns, self.user_ns)
 (warnings.py:99)
[tardis.io.atom_data.base][INFO   ]  Read Atom Data with UUID=6f7b09e887a311e7a06b246e96350010 and MD5=864f1753714343c41f99cb065710cace. (base.py:184)
[tardis.io.atom_data.base][INFO   ]  Non provided atomic data: synpp_refs, photoionization_data (base.py:187)
[tardis.simulation.base][INFO   ]  Starting iteration 1/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 7.96915e+42 

[tardis.simulation.base][INFO   ]  t_inner 10123.177 K -- next t_inner 11180.783 K (base.py:352)
[tardis.simulation.base][INFO   ]  Starting iteration 10/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.28170e+43 erg / s Luminosity absorbed = 4.19511e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:359)
[tardis.simulation.base][INFO   ]  Plasma stratification:
	              t_rad    next_t_rad         w    next_w
	Shell                                                
	0      10666.223660  11427.410445  0.445097  0.500788
	5      11000.288585  11612.349822  0.163424  0.193129
	10     10822.884326  11373.245169  0.101641  0.121622
	15     10384.181273  11047.955461  0.078432  0.090937

 (base.py:350)
[tardis.simulation.base][INFO   ]  t_inner 11180.783 K -- next t_inner 10164.432 K (base.py:352)
[tardis.simulation.base][INFO   ]  Starting iteration 11/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 8.79074e+42 erg / s Lum

[tardis.simulation.base][INFO   ]  t_inner 10314.623 K -- next t_inner 11009.102 K (base.py:352)
[tardis.simulation.base][INFO   ]  Starting iteration 20/20 (base.py:268)
[tardis.simulation.base][INFO   ]  Luminosity emitted = 1.21062e+43 erg / s Luminosity absorbed = 3.88405e+42 erg / s Luminosity requested = 1.05928e+43 erg / s (base.py:359)
[tardis.simulation.base][INFO   ]  Simulation finished in 20 iterations and took 49.39 s (base.py:308)


## Data exploration in old GUI code

In [5]:
from astropy import units as u
from tardis.analysis import LastLineInteraction
from tardis.util.base import species_tuple_to_string

atom_data = sim.plasma.atomic_data.atom_data
lines_data = sim.plasma.lines
line_interaction_analysis = LastLineInteraction.from_model(sim)


def select_in_line_interaction_analysis(wavelength_start, wavelength_end, packet_filter_mode='packet_nu'):
    # LastLineInteraction class's wavelength parameter setters slice the entire lines_data each time
    # (so modify object in global scope and later in lineinfo class scope)

    # Better approach: do it locally but how can it work with current structure of LastLineInteraction?
    line_interaction_analysis.packet_filter_mode = 'packet_nu'
    line_interaction_analysis.wavelength_start = wavelength_start * u.angstrom
    line_interaction_analysis.wavelength_end = wavelength_end * u.angstrom

    # WARNING: Time Bottleneck is this function - optimize way of slicing lines_data as per wavelength
    # see update_last_interaction_filter function in analysis.py

### Species Abundances Table

In [6]:
select_in_line_interaction_analysis(5000, 10000)

In [30]:
selected_species_group = line_interaction_analysis.last_line_in.groupby(
    ['atomic_number', 'ion_number'])
selected_species_symbols = [species_tuple_to_string(
    item) for item in selected_species_group.groups.keys()]
selected_species_symbols

['O I', 'Mg II', 'Si II', 'Si III', 'S II', 'Ar I', 'Ar II', 'Ca II']

In [52]:
selected_species_group.size(), selected_species_group.wavelength.count()

(atomic_number  ion_number
 8              0               23
 12             1               23
 14             1             1084
                2              461
 16             1             2960
 18             0                2
                1              135
 20             1              176
 dtype: int64,
 atomic_number  ion_number
 8              0               23
 12             1               23
 14             1             1084
                2              461
 16             1             2960
 18             0                2
                1              135
 20             1              176
 Name: wavelength, dtype: int64)

In [53]:
line_interaction_analysis.last_line_in.shape[0], line_interaction_analysis.last_line_in.wavelength.count(
)

(4864, 4864)

In [56]:
selected_species_abundances = selected_species_group.size(
)/line_interaction_analysis.last_line_in.shape[0]
selected_species_abundances

atomic_number  ion_number
8              0             0.004729
12             1             0.004729
14             1             0.222862
               2             0.094778
16             1             0.608553
18             0             0.000411
               1             0.027755
20             1             0.036184
dtype: float64

In [67]:
selected_species_abundances.index = pd.Index(
    selected_species_symbols, name='Species')
selected_species_abundances.name = 'Fractional Abundance'
selected_species_abundances.sort_values(ascending=False).to_frame()

,Fractional Abundance
Species,
S II,0.608553
Si II,0.222862
Si III,0.094778
Ca II,0.036184
Ar II,0.027755
Mg II,0.004729
O I,0.004729
Ar I,0.000411


In [69]:
# Putting all together
def species_abundances_in_selected_wavelengths(wavelength_start, wavelength_end, packet_filter_mode='packet_nu'):
    select_in_line_interaction_analysis(
        wavelength_start, wavelength_end, packet_filter_mode)

    selected_species_group = line_interaction_analysis.last_line_in.groupby(
        ['atomic_number', 'ion_number'])
    selected_species_symbols = [species_tuple_to_string(
        item) for item in selected_species_group.groups.keys()]

    selected_species_abundances = selected_species_group.size(
    )/line_interaction_analysis.last_line_in.shape[0]
    selected_species_abundances.index = pd.Index(
        selected_species_symbols, name='Species')
    selected_species_abundances.name = 'Fractional Abundance'
    return selected_species_abundances.sort_values(ascending=False).to_frame()

In [70]:
species_abundances_in_selected_wavelengths(5000, 10000)

,Fractional Abundance
Species,
S II,0.608553
Si II,0.222862
Si III,0.094778
Ca II,0.036184
Ar II,0.027755
Mg II,0.004729
O I,0.004729
Ar I,0.000411


#### Benchmarking

In [71]:
%lprun - f species_abundances_in_selected_wavelengths species_abundances_in_selected_wavelengths(5000, 10000)

In [74]:
%lprun - f select_in_line_interaction_analysis select_in_line_interaction_analysis(5000, 10000)

### Line count table

In [77]:
from tardis.util.base import species_string_to_tuple

selected_species = 'Si II'
selected_species_tuple = species_string_to_tuple(selected_species)

In [79]:
current_last_line_in = line_interaction_analysis.last_line_in.xs(
    key=(selected_species_tuple[0], selected_species_tuple[1]),
    level=['atomic_number', 'ion_number'],
    drop_level=False).reset_index()
current_last_line_in

,atomic_number,ion_number,level_number_lower,level_number_upper,line_id,wavelength,f_ul,f_lu,nu,B_lu,B_ul,A_ul,wavelength_cm
0,14,1,15,31,65301,3210.953456,0.072753,0.109129,9.336556e+14,5.883145e+09,3.922097e+09,4.706770e+07,0.000032
1,14,1,15,22,65409,5057.393754,0.652903,0.979355,5.927805e+14,8.315767e+10,5.543845e+10,1.702697e+08,0.000051
2,14,1,11,23,65355,4132.059299,0.374033,0.498711,7.255280e+14,3.459799e+10,2.594849e+10,1.461232e+08,0.000041
3,14,1,5,13,65328,3863.690217,0.076203,0.038101,7.759226e+14,2.471598e+09,4.943196e+09,3.404926e+07,0.000039
4,14,1,0,7,65082,1526.707000,0.129113,0.129113,1.963654e+15,3.309491e+09,3.309491e+09,3.694884e+08,0.000015
5,14,1,6,15,65327,3857.111505,0.069333,0.046222,7.772460e+14,2.993274e+09,4.489911e+09,3.108550e+07,0.000039
6,14,1,5,27,65114,1870.230000,0.106645,0.106645,1.602971e+15,3.348656e+09,3.348656e+09,2.033725e+08,0.000019
7,14,1,1,7,65083,1533.431000,0.257614,0.128807,1.955044e+15,3.316193e+09,6.632386e+09,7.307751e+08,0.000015
8,14,1,5,13,65328,3863.690217,0.076203,0.038101,7.759226e+14,2.471598e+09,4.943196e+09,3.404926e+07,0.000039
9,14,1,13,20,65480,5959.209525,0.250017,0.250017,5.030742e+14,2.501464e+10,2.501464e+10,4.696074e+07,0.000060


In [81]:
current_last_line_out = line_interaction_analysis.last_line_out.xs(
    key=(selected_species_tuple[0], selected_species_tuple[1]),
    level=['atomic_number', 'ion_number'],
    drop_level=False).reset_index()

current_last_line_out

,atomic_number,ion_number,level_number_lower,level_number_upper,line_id,wavelength,f_ul,f_lu,nu,B_lu,B_ul,A_ul,wavelength_cm
0,14,1,13,20,65480,5959.209525,0.250017,0.250017,5.030742e+14,2.501464e+10,2.501464e+10,4.696074e+07,0.000060
1,14,1,15,22,65409,5057.393754,0.652903,0.979355,5.927805e+14,8.315767e+10,5.543845e+10,1.702697e+08,0.000051
2,14,1,23,37,65572,9415.247525,0.954993,1.193741,3.184117e+14,1.887023e+11,1.509619e+11,7.185860e+07,0.000094
3,14,1,7,13,65498,6373.132547,0.496558,0.496558,4.704005e+14,5.313228e+10,5.313228e+10,8.154669e+07,0.000064
4,14,1,7,15,65494,6348.864031,0.495382,0.990764,4.721986e+14,1.056092e+11,5.280458e+10,8.197665e+07,0.000063
5,14,1,7,13,65498,6373.132547,0.496558,0.496558,4.704005e+14,5.313228e+10,5.313228e+10,8.154669e+07,0.000064
6,14,1,7,13,65498,6373.132547,0.496558,0.496558,4.704005e+14,5.313228e+10,5.313228e+10,8.154669e+07,0.000064
7,14,1,7,15,65494,6348.864031,0.495382,0.990764,4.721986e+14,1.056092e+11,5.280458e+10,8.197665e+07,0.000063
8,14,1,7,13,65498,6373.132547,0.496558,0.496558,4.704005e+14,5.313228e+10,5.313228e+10,8.154669e+07,0.000064
9,14,1,7,13,65498,6373.132547,0.496558,0.496558,4.704005e+14,5.313228e+10,5.313228e+10,8.154669e+07,0.000064


In [82]:
current_last_line_in.line_id.unique()

array([65301, 65409, 65355, 65328, 65082, 65327, 65114, 65083, 65480,
       65109, 65113, 65263, 65498, 65134, 65244, 65101, 65108, 65494,
       65141, 65408, 64963, 64958, 65143, 65229, 65410, 65481, 65587,
       65027, 65133, 65072, 65343, 65076, 64964, 65107, 65102, 65574,
       65011, 65012, 65572, 65354, 65247, 64959, 65326, 64957, 65085,
       64977, 65173, 65234, 65543, 65353, 65345, 65164, 65010, 65026,
       65077, 65261, 65242, 65222, 65250, 65070, 65041, 65084, 65032,
       64930, 65144, 64929, 65132, 65447, 65052, 65298, 64932, 65237,
       65369, 65445, 65562, 65545, 65086, 65246, 65051, 64978, 64980,
       65489, 65604, 64934, 65170, 65100, 65067, 65115, 65209, 64918,
       65171, 65112, 65313, 65665, 65166, 65364, 65632])

In [83]:
current_last_line_out.line_id.unique()

array([65480, 65409, 65572, 65498, 65494, 65516, 65481, 65543, 65545,
       65408, 65489, 65574, 65410, 65447, 65544, 65525, 65469, 65577,
       65582, 65505, 65416, 65444, 65519, 65491, 65536, 65550])

In [84]:
current_last_line_in['line_id_out'] = current_last_line_out.line_id
# Q: why, are they really corresponding?

In [87]:
last_line_in_string = []
last_line_count = []
grouped_line_interactions = current_last_line_in.groupby(
    ['line_id', 'line_id_out'])
exc_deexc_string = 'exc. %d-%d (%.2f A) de-exc. %d-%d (%.2f A)'

In [94]:
# all(grouped_line_interactions.wavelength.count() == grouped_line_interactions.size())

True

In [95]:
grouped_line_interactions.size()

line_id  line_id_out
64918    65494           1
64929    65494           1
64930    65498           1
64932    65498           1
64934    65498           1
64957    65410           1
         65494           2
         65498           1
64958    65409           2
64959    65408           1
         65498           1
64963    65480           3
         65481           9
         65494           2
         65498           1
64964    65480           4
         65481           2
         65494           4
         65498           3
64977    65481           1
64978    65494           1
64980    65498           1
65010    65481           1
         65494           1
         65498           2
         65574           1
65011    65494           1
         65505           1
         65543           1
         65545           1
                        ..
65481    65498           9
         65543           1
65489    65494           1
         65545           1
65494    65408           1
       

In [97]:
lines_data = lines_data.reset_index().set_index('line_id')
lines_data

,atomic_number,ion_number,level_number_lower,level_number_upper,wavelength,f_ul,f_lu,nu,B_lu,B_ul,A_ul,wavelength_cm
line_id,,,,,,,,,,,,
67813,14,5,0,36,66.772000,0.027030,0.040545,4.489793e+16,4.545377e+07,3.030251e+07,4.043917e+10,6.677200e-07
67814,14,5,0,35,69.204000,0.086684,0.086684,4.332011e+16,1.007179e+08,1.007179e+08,1.207313e+11,6.920400e-07
67815,14,5,0,34,69.236000,0.061923,0.092884,4.330008e+16,1.079711e+08,7.198073e+07,8.616424e+10,6.923600e-07
67816,14,5,1,35,69.448000,0.017699,0.035397,4.316790e+16,4.127293e+07,2.063646e+07,2.447726e+10,6.944800e-07
67817,14,5,0,33,71.181000,0.091590,0.137385,4.211692e+16,1.641873e+08,1.094582e+08,1.205765e+11,7.118100e-07
67818,14,5,0,32,71.273000,0.000755,0.000755,4.206256e+16,9.034426e+05,9.034426e+05,9.913617e+08,7.127300e-07
67819,14,5,0,31,71.302000,0.014756,0.007378,4.204545e+16,8.832370e+06,1.766474e+07,1.936015e+10,7.130200e-07
67820,14,5,1,32,71.533000,0.077257,0.154515,4.190967e+16,1.855718e+08,9.278590e+07,1.007092e+11,7.153300e-07
67821,14,5,1,31,71.561000,0.026240,0.026240,4.189327e+16,3.152695e+07,3.152695e+07,3.417901e+10,7.156100e-07


In [102]:
for line_id, count in grouped_line_interactions.size().iteritems():
    current_line_in = lines_data.loc[line_id[0]]
    current_line_out = lines_data.loc[line_id[1]]
    last_line_in_string.append(exc_deexc_string % (
        current_line_in['level_number_lower'],
        current_line_in['level_number_upper'],
        current_line_in['wavelength'],
        current_line_out['level_number_upper'],a
        current_line_out['level_number_lower'],
        current_line_out['wavelength']))
    last_line_count.append(count)

In [104]:
last_line_in_string

['exc. 1-58 (820.50 A) de-exc. 15-7 (6348.86 A)',
 'exc. 1-45 (850.14 A) de-exc. 15-7 (6348.86 A)',
 'exc. 0-30 (889.72 A) de-exc. 13-7 (6373.13 A)',
 'exc. 1-30 (892.00 A) de-exc. 13-7 (6373.13 A)',
 'exc. 1-29 (901.74 A) de-exc. 13-7 (6373.13 A)',
 'exc. 0-21 (989.87 A) de-exc. 21-15 (5057.73 A)',
 'exc. 0-21 (989.87 A) de-exc. 15-7 (6348.86 A)',
 'exc. 0-21 (989.87 A) de-exc. 13-7 (6373.13 A)',
 'exc. 1-22 (992.68 A) de-exc. 22-15 (5057.39 A)',
 'exc. 1-21 (992.70 A) de-exc. 21-13 (5042.43 A)',
 'exc. 1-21 (992.70 A) de-exc. 13-7 (6373.13 A)',
 'exc. 0-20 (1020.70 A) de-exc. 20-13 (5959.21 A)',
 'exc. 0-20 (1020.70 A) de-exc. 20-15 (5980.59 A)',
 'exc. 0-20 (1020.70 A) de-exc. 15-7 (6348.86 A)',
 'exc. 0-20 (1020.70 A) de-exc. 13-7 (6373.13 A)',
 'exc. 1-20 (1023.70 A) de-exc. 20-13 (5959.21 A)',
 'exc. 1-20 (1023.70 A) de-exc. 20-15 (5980.59 A)',
 'exc. 1-20 (1023.70 A) de-exc. 15-7 (6348.86 A)',
 'exc. 1-20 (1023.70 A) de-exc. 13-7 (6373.13 A)',
 'exc. 0-18 (1190.42 A) de-exc. 20-

In [111]:
line_counts = pd.DataFrame(last_line_count, index=last_line_in_string)
line_counts.columns = ['Num. pkts {}'.format(len(current_last_line_in))]
line_counts

,Num. pkts 1084
exc. 1-58 (820.50 A) de-exc. 15-7 (6348.86 A),1
exc. 1-45 (850.14 A) de-exc. 15-7 (6348.86 A),1
exc. 0-30 (889.72 A) de-exc. 13-7 (6373.13 A),1
exc. 1-30 (892.00 A) de-exc. 13-7 (6373.13 A),1
exc. 1-29 (901.74 A) de-exc. 13-7 (6373.13 A),1
exc. 0-21 (989.87 A) de-exc. 21-15 (5057.73 A),1
exc. 0-21 (989.87 A) de-exc. 15-7 (6348.86 A),2
exc. 0-21 (989.87 A) de-exc. 13-7 (6373.13 A),1
exc. 1-22 (992.68 A) de-exc. 22-15 (5057.39 A),2
exc. 1-21 (992.70 A) de-exc. 21-13 (5042.43 A),1


### Putting it all

In [113]:
def last_line_counts_of_species(selected_species):
    selected_species_tuple = species_string_to_tuple(selected_species)

    current_last_line_in = line_interaction_analysis.last_line_in.xs(
        key=(selected_species_tuple[0], selected_species_tuple[1]),
        level=['atomic_number', 'ion_number'],
        drop_level=False).reset_index()

    current_last_line_out = line_interaction_analysis.last_line_out.xs(
        key=(selected_species_tuple[0], selected_species_tuple[1]),
        level=['atomic_number', 'ion_number'],
        drop_level=False).reset_index()

    current_last_line_in['line_id_out'] = current_last_line_out.line_id

    last_line_in_string = []
    last_line_count = []
    grouped_line_interactions = current_last_line_in.groupby(
        ['line_id', 'line_id_out'])
    exc_deexc_string = 'exc. %d-%d (%.2f A) de-exc. %d-%d (%.2f A)'

    for line_id, count in grouped_line_interactions.size().iteritems():
        current_line_in = lines_data.loc[line_id[0]]
        current_line_out = lines_data.loc[line_id[1]]
        last_line_in_string.append(exc_deexc_string % (
            current_line_in['level_number_lower'],
            current_line_in['level_number_upper'],
            current_line_in['wavelength'],
            current_line_out['level_number_upper'],
            current_line_out['level_number_lower'],
            current_line_out['wavelength']))
        last_line_count.append(count)

    line_counts = pd.DataFrame(last_line_count, index=last_line_in_string)
    line_counts.columns = ['Num. pkts {}'.format(len(current_last_line_in))]
    return line_counts

In [114]:
last_line_counts_of_species('Si II')

,Num. pkts 1084
exc. 1-58 (820.50 A) de-exc. 15-7 (6348.86 A),1
exc. 1-45 (850.14 A) de-exc. 15-7 (6348.86 A),1
exc. 0-30 (889.72 A) de-exc. 13-7 (6373.13 A),1
exc. 1-30 (892.00 A) de-exc. 13-7 (6373.13 A),1
exc. 1-29 (901.74 A) de-exc. 13-7 (6373.13 A),1
exc. 0-21 (989.87 A) de-exc. 21-15 (5057.73 A),1
exc. 0-21 (989.87 A) de-exc. 15-7 (6348.86 A),2
exc. 0-21 (989.87 A) de-exc. 13-7 (6373.13 A),1
exc. 1-22 (992.68 A) de-exc. 22-15 (5057.39 A),2
exc. 1-21 (992.70 A) de-exc. 21-13 (5042.43 A),1


In [115]:
lli = LastLineInteraction.from_model(sim)
lli

In [116]:
%lprun -f LastLineInteraction.from_model LastLineInteraction.from_model(sim)

## Interact with IntRangeSlider using qgrid table widget

In [6]:
import ipywidgets as ipw
initial_range = [5000, 10000]
wavelength_slider = ipw.IntRangeSlider(
    value=initial_range,
    min=0,
    max=30000,  # what is ideal - infinity or something else?
    step=1,
    description='Wavelength:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
wavelength_slider.layout.width = '350px'
# wavelength_slider

In [7]:
import qgrid
table = qgrid.show_grid(line_info(sim, *initial_range))
table.layout.width = '500px'
# table

In [8]:
def on_value_change(change):
    table.df = line_info(sim, *change['new'])


wavelength_slider.observe(on_value_change, names='value')

display(wavelength_slider, table)

IntRangeSlider(value=(5000, 10000), continuous_update=False, description='Wavelength:', layout=Layout(width='3…

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Testing other ways to interact with df

In [13]:
out = ipw.Output(layout={'border': '1px solid yellow'})
out

Output(layout=Layout(border='1px solid yellow'))

In [19]:
line_info(sim, *(1000, 2000))

,Species,Species Abundances
0,O I,0.006929
1,Mg II,0.003234
2,Si II,0.478134
3,Si III,0.382199
4,Si IV,0.022174
5,S I,0.000308
6,S II,0.052510
7,S III,0.019711
8,S IV,0.001848
9,Ar I,0.000462


In [32]:
wavelength_slider.unobserve_all()

In [31]:
def update_line_info(wavelength_slider):
    #     return line_info(sim, *wavelength_slider.value)
    return wavelength_slider


ipw.interact(update_line_info, wavelength_slider)

TypeError: __call__() takes from 1 to 2 positional arguments but 3 were given

In [ ]:
class LineInteractionTables(QtWidgets.QWidget):
    """Widget to hold the line interaction tables used by
    LineInfo which in turn is used by spectrum widget.

    """

    def __init__(self, line_interaction_analysis, atom_data, lines_data, description,
                 tablecreator):
        """Create the widget and set data."""
#         super(LineInteractionTables, self).__init__()
#         self.createTable = tablecreator
#         self.text_description = QtWidgets.QLabel(str(description))
#         self.species_table = QtWidgets.QTableView()
#         self.transitions_table = QtWidgets.QTableView()
#         self.layout = QtWidgets.QHBoxLayout()
        self.line_interaction_analysis = line_interaction_analysis
        self.atom_data = atom_data
        self.lines_data = lines_data.reset_index().set_index('line_id')
        line_interaction_species_group = \
            line_interaction_analysis.last_line_in.groupby(['atomic_number',
                                                            'ion_number'])
        self.species_selected = sorted(
            line_interaction_species_group.groups.keys())
        species_symbols = [tardis.util.base.species_tuple_to_string(
            item) for item in self.species_selected]
        species_table_model = self.createTable([species_symbols, ['Species']])
        species_abundances = (
            line_interaction_species_group.wavelength.count().astype(float) /
            line_interaction_analysis.last_line_in.wavelength.count()).astype(float).tolist()
        species_abundances = list(map(float, species_abundances))
        species_table_model.add_data(species_abundances)
        self.species_table.setModel(species_table_model)

        line_interaction_species_group.wavelength.count()
        self.layout.addWidget(self.text_description)
        self.layout.addWidget(self.species_table)
        self.sectionClicked = QtCore.Signal(int)
        self.species_table.verticalHeader().sectionClicked.connect(self.on_species_clicked)
        self.layout.addWidget(self.transitions_table)

        self.setLayout(self.layout)
        self.show()

    def on_species_clicked(self, index):
        """"""
        current_species = self.species_selected[index]
        last_line_in = self.line_interaction_analysis.last_line_in
        last_line_out = self.line_interaction_analysis.last_line_out

        current_last_line_in = last_line_in.xs(
            key=(current_species[0], current_species[1]),
            level=['atomic_number', 'ion_number'],
            drop_level=False).reset_index()
        current_last_line_out = last_line_out.xs(
            key=(current_species[0], current_species[1]),
            level=['atomic_number', 'ion_number'],
            drop_level=False).reset_index()

        current_last_line_in['line_id_out'] = current_last_line_out.line_id

        last_line_in_string = []
        last_line_count = []
        grouped_line_interactions = current_last_line_in.groupby(
            ['line_id', 'line_id_out'])
        exc_deexc_string = 'exc. %d-%d (%.2f A) de-exc. %d-%d (%.2f A)'

        for line_id, row in grouped_line_interactions.wavelength.count().iteritems():
            current_line_in = self.lines_data.loc[line_id[0]]
            current_line_out = self.lines_data.loc[line_id[1]]
            last_line_in_string.append(exc_deexc_string % (
                current_line_in['level_number_lower'],
                current_line_in['level_number_upper'],
                current_line_in['wavelength'],
                current_line_out['level_number_upper'],
                current_line_out['level_number_lower'],
                current_line_out['wavelength']))
            last_line_count.append(int(row))

        last_line_in_model = self.createTable([last_line_in_string, [
            'Num. pkts %d' % current_last_line_in.wavelength.count()]])
        last_line_in_model.add_data(last_line_count)
        self.transitions_table.setModel(last_line_in_model)